!pip install --upgrade "ibm-watson>=7.0.1"

# Description

The goal of these notebook is to sucessfully download the actions and upload them witht he new Custom
extension ids in case we modified them for some reason. 



In [3]:
import pprint
from dotenv import load_dotenv
import os
load_dotenv()

True

# Assistant setup 

In [4]:
URL = 'https://api.us-east.assistant.watson.cloud.ibm.com/'
API_KEY = os.getenv(key='API_KEY')

# put here the name of the assistant to replace extensions ids 
ASSISTANT_NAME = 'Experiments_Ask_Proc_Main'

In [69]:
# Auhentification 
import json
from ibm_watson import AssistantV2
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator(API_KEY)
assistant = AssistantV2(
    version='2021-06-14',
    authenticator = authenticator
)

assistant.set_service_url(URL)



# Changes setup 
In our case we only have 2 custom expensions and only one is the most commun to have changes so that is why I have to flows 
One when I only want to change the ids for EPM api And other when I want to change epm and watsonx.ai ids 


In [70]:

# change to False if you will modify epm and watsonx apis 
ONLY_EPM_API_CHANGE = True

# Old epm api ids 
old_epm_catalog_ids = [ 
    "e4e0380c-e956-4b16-8b8d-c8b61235c8aa"
]

old_epm_hash_ids = [
    "b86af2fd95b085f30eee5e7aa7767f2353c6742aa494bcce7d6d33da9eb83f05"
]


new_epm_catalog_id = "9a6ab4aa-b6dd-47f4-a8f8-00b11ab6c053"
new_epm_hash_id = "1f4f904c10572aff695fc0b2c053edb0824a7b6b70d457f8ec6ac3a228607885"


# old watsonx.ai ids 
old_watsonx_catalog_id = [
    "2a9c63f0-5e9d-44a8-9c6e-92a5684030be"
    ]
    
old_watsonx_hash_id = [
    "cc50493e455de183d661b95360196ddb468a02051add6dcaf4a6479b34900be5"
]


new_watsonx_catalog_id = "f6357af0-db6b-402e-a61a-915577e4e351"
new_watsonx_hash_id = "cc50493e455de183d661b95360196ddb468a02051add6dcaf4a6479b34900be5"


# Getting the assistant ID using the assistant name

In [71]:
assistant_list = assistant.list_assistants()

In [72]:
# find your assistant by name 


for bot in assistant_list.get_result().get('assistants'):
    if bot.get('name') == ASSISTANT_NAME:
        ASSISTANT_ID = bot.get('assistant_id')
        pprint.pprint(bot)

print(f'\n\nYour assistant id is: {ASSISTANT_ID}')

{'assistant_environments': [{'environment': 'live',
                             'environment_id': '3d6221bd-3902-437b-b8fe-61659530d549',
                             'name': 'live'},
                            {'environment': 'draft',
                             'environment_id': '8b2d321a-d698-42d5-a8a6-afadb7630aac',
                             'name': 'draft'}],
 'assistant_id': 'bf5deaec-5f82-47a7-8d75-6b843b3190af',
 'assistant_skills': [{'skill_id': '9a0cd298-cf7c-4bc1-b7a2-3669759ee53e',
                       'type': 'dialog'},
                      {'skill_id': '8ed9e13a-7bc1-4ae0-babd-06080309ad90',
                       'type': 'action'}],
 'description': '',
 'language': 'en',
 'multi_lingual': {'enabled': False},
 'name': 'Experiments_Ask_Proc_Main'}


Your assistant id is: bf5deaec-5f82-47a7-8d75-6b843b3190af


# Get the skills data from your assistant.

In [73]:
'''
A successful call to this method only initiates an asynchronous export. 
The exported JSON data is not available until processing completes.

After the initial request is submitted, you can poll the status of the operation by calling the same request again and 
checking the value of the status property. If an error occurs (indicated by a status value of Failed), 
the status_description property provides more information about the error, and the status_errors property 
contains an array of error messages that caused the failure.

When processing has completed, the request returns the exported JSON data. Remember that the usual rate limits apply.
'''

skills = assistant.export_skills(assistant_id=ASSISTANT_ID)

In [74]:
# here you get the json data 
assistand_data = skills.result


In [ ]:
assistand_data

In [75]:
assistand_data.keys()

dict_keys(['assistant_skills', 'assistant_state'])

# Transform the assistant_skills data into text 

In [77]:
assistant_skills_txt = str(assistand_data['assistant_skills'])

# Replace in the string

With the data in string format you can now use the replace method to replace the catalog and hash ids 

In [78]:
# chnage epm api 
if ONLY_EPM_API_CHANGE == True:
    for old_catalog_id in old_epm_catalog_ids:
        assistant_skills_txt = assistant_skills_txt.replace(old_catalog_id, new_epm_catalog_id)

    for old_hash_id in old_epm_hash_ids:
        assistant_skills_txt = assistant_skills_txt.replace(old_hash_id, new_epm_hash_id)

if ONLY_EPM_API_CHANGE == False:
    # epm changes 
    for old_catalog_id in old_epm_catalog_ids:
        assistant_skills_txt = assistant_skills_txt.replace(old_catalog_id, new_epm_catalog_id)

    for old_hash_id in old_epm_hash_ids:
        assistant_skills_txt = assistant_skills_txt.replace(old_hash_id, new_epm_hash_id)

    # watsonx.ai changes 
    for old_catalog_id in old_watsonx_catalog_id:
        assistant_skills_txt = assistant_skills_txt.replace(old_catalog_id, new_watsonx_catalog_id)

    for old_hash_id in old_watsonx_hash_id:
        assistant_skills_txt = assistant_skills_txt.replace(old_hash_id, new_watsonx_hash_id)
    

# After replacement 

Once the ids are replaced , now we can transform the text again to a list using eval function. 

In [79]:
# then we can go back to list here.
assistant_skills_list = eval(assistant_skills_txt)

# Upload updated skills 

Now we can upload the skills data again in the server 

In [80]:
import_skills = assistant.import_skills(assistant_id=ASSISTANT_ID, 
                                        include_audit=True, 
                                        assistant_skills=assistant_skills_list, 
                                        assistant_state = assistand_data['assistant_state'])

# Check the inport state 

In [84]:
assistant.import_skills_status(assistant_id=ASSISTANT_ID).get_result()

{'status': 'Completed', 'assistant_id': 'bf5deaec-5f82-47a7-8d75-6b843b3190af'}